In [ ]:
import os
import sys
import csv
import re
import jieba
import pandas as pd # 引用套件並縮寫為 pd  

from jieba.analyse import *

filename='ptt20180315-3yv2utf8.csv'
datapath = os.getcwd() + '\\'+ filename
setdictpath = os.getcwd() + '\\'+ 'dict.txt.big'
userdictpath  = os.getcwd() + '\\'+ 'user_dict2.txt'
stopword_path= os.getcwd() + '\\'+ 'stopword_tw3.txt'
#outputpath = os.getcwd() + '\\'+ 'output_tfidf.csv'

#dictionary path assign
jieba.set_dictionary(setdictpath)
jieba.load_userdict(userdictpath) 

#content as a list
def sectencelist(corpus):
    sentence = []
    for s in corpus :
        sentence.append(s)
    return sentence

# make stopword list
def stopwordslist(stopword_path):  
    stopwords = [line.strip() for line in open(stopword_path, 'r', encoding='utf-8').readlines()]  
    return stopwords 

#去除數字  標點符號
def symbolremove(list):
    sentence_re = []
    for s in list:
        s = re.sub(r'[A-Za-z]+','', s)
        s = re.sub(r'\d+', '', s)
        s = re.sub('<[^>]*>', '', s)
        s = re.sub('[\W]+', '', s)
        sentence_re.append(s)
    return sentence_re

#用 jieba 斷詞
def clearText(text):
    mywordlist = []
    seg_list = jieba.cut(text, cut_all=False)
    liststr="/ ".join(seg_list)
    f_stop = open(stopword_path, encoding='utf-8')
    try:
        f_stop_text = f_stop.read( )
        #f_stop_text=unicode(f_stop_text,'utf-8')
    finally:
        f_stop.close( )
    f_stop_seg_list=f_stop_text.split('\n')
    for myword in liststr.split('/'):
        if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
            mywordlist.append(myword)
    return ''.join(mywordlist)   
    
    

df = pd.read_csv(datapath ,encoding='UTF-8')

corpus = df.values[:,3]

step1 = sectencelist(corpus)
step2 = symbolremove(step1)
step3_lists =[]
for i in range(0,len(step2)):
    step3_list=clearText(step2[i])
    step3_lists.append(step3_list)

In [ ]:
with open(stopword_path,encoding='utf8') as f:
    stop_list = [i.strip() for i in f.readlines()]

In [ ]:
from  sklearn.feature_extraction.text  import  TfidfTransformer  
from  sklearn.feature_extraction.text  import  CountVectorizer

transformer = TfidfTransformer()
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(step3_lists)
word = vectorizer.get_feature_names()
tfidf = transformer.fit_transform(X) 
weight = tfidf.toarray()
final_data_list=[]
for i in range(len(weight)):#打印每类文本的tf-idf词语权重，第一个for遍历所有文本，第二个for便利某一类文本下的词语权重  
    #print (u"-------这里输出第",i,u"类文本的词语tf-idf权重------" )
    #print(weight)
    data_list=[]
    for j in range(len(word)):
        data={}
        data['word']=word[j]
        data['weight']=weight[i][j]
        data_list.append(data)  
    sort_list=sorted(data_list, key=lambda k: k['weight'],reverse=True)
    #sort words by weight
    clean_data_list=[]
    for words in sort_list[0:30]:
        clean_data={}
        if words['weight'] == 0:
            break
        clean_data['word']=words['word']
        clean_data['weight']=words['weight'].astype(type('float', (float,), {}))
        clean_data_list.append(clean_data)
    final_data_list.append(clean_data_list)

In [ ]:
final_data_list

In [ ]:
data_list=[]
for i in range(len(step3_lists)):
    data={}
    data['words']=final_data_list[i]
    data['id']=df.values[i,0]
    data_list.append(data)
data_list

In [ ]:
tag_words_list=[]
for i in data_list:
    tag_words=[]
    for i in i['words']:
        tag_words.append(i['word'])
    tag_words_list.append(' '.join(tag_words))
tag_words_list

In [ ]:
final_data_list=0

In [ ]:
from  sklearn.feature_extraction.text  import  TfidfTransformer  
from  sklearn.feature_extraction.text  import  CountVectorizer
transformer = TfidfTransformer() 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tag_words_list)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
distortions = []
for i in range(1, 15):
    km = KMeans(n_clusters=i, 
                init='k-means++', 
                n_init=10, 
                max_iter=300, 
                random_state=0)
    km.fit(X)
    distortions.append(km.inertia_)
    result=km.fit_predict(X)
    km.inertia_
    print('predict : ',result)
plt.plot(range(1, 15), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
plt.show()

In [ ]:
result[0]

In [ ]:
km = KMeans(n_clusters=11, 
            init='k-means++', 
            n_init=10, 
            max_iter=300, 
            random_state=0)
km.fit(X)
distortions.append(km.inertia_)
result=km.fit_predict(X)
print('predict : ',result)

In [ ]:
tag_words_list=0
distortions=0
cut_article_comment_list=0

In [ ]:
x=0

In [ ]:
new_data_list=[]
for i in range(len(data_list)):
    data={}
    data['group'] = result[i].astype(type('int', (int,), {}))
    data['id']= df.values[i,0]
    data['words']=data_list[i]['words']
    new_data_list.append(data)
new_data_list

In [ ]:
#export result to csv file
outputpath = os.getcwd() + '\\'+ 'kmeansgrouping_result.csv'

with open(outputpath, 'w', encoding='UTF-8') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(new_data_list)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words=stop_list)
X = vectorizer.fit_transform(step3_lists)

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter

k = 15
km = KMeans(n_clusters=k, random_state=2)
km.fit(X)
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
label_count = Counter(km.labels_)



for i in range(k):
    print("Topic %d (Count: %d):" % (i, label_count[i]), end=" ")
    for ind in order_centroids[i, :10]:
        print('%s ' % terms[ind], end=" ")
    print() 

In [ ]:
for i in range(k):
    word_score = []
    for ind, score in zip(order_centroids[i, :10],  km.cluster_centers_[i, ::-1]):
        word_score.append((terms[ind], score))
        
    word_wc = WordCloud(font_path="data/msjh.ttf").generate_from_frequencies(''.join(word_score))
    plt.imshow(word_wc)
    plt.axis("off")
    plt.title("Topic %d (Count: %d):" % (i, label_count[i]))
    plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
distortions = []
for i in range(1, 15):
    km = KMeans(n_clusters=i, 
                init='k-means++', 
                n_init=10, 
                max_iter=300, 
                random_state=0)
    km.fit(X)
    distortions.append(km.inertia_)
    result=km.fit_predict(X)
    km.inertia_
    print('predict : ',result)
plt.plot(range(1, 15), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
#如何選擇 k

#實務上我們讓程式幫忙選擇一個適合的 k，使得群間差異最大、群內差異最小

from sklearn import cluster, datasets, metrics
import matplotlib.pyplot as plt

# 迴圈
silhouette_avgs = []
ks = range(2, 20)
for k in ks:
    kmeans_fit = cluster.KMeans(n_clusters = k).fit(X)
    cluster_labels = kmeans_fit.labels_
    silhouette_avg = metrics.silhouette_score(X, cluster_labels)
    silhouette_avgs.append(silhouette_avg)

# 作圖並印出 k = 2 到 20 的績效
plt.bar(ks, silhouette_avgs)
plt.show()

In [ ]:
print(silhouette_avgs)

In [ ]:
#由績效圖我們發現 在2群、3群的績效比較好，因此我們可以將K設定為2 OR 3

km = KMeans(n_clusters=2)  #K=2群
y_pred = km.fit_predict(X)
plt.figure(figsize=(10, 6))
plt.xlabel('Salary')
plt.ylabel('Rate of working')
plt.scatter(X[:, 0], X[:, 1], c=y_pred) #C是第三維度 已顏色做維度
plt.show()

km.cluster_centers_ #各群中心點(X,Y)的位置


In [ ]:
!pip install wordcloud

In [ ]:
#利用文字雲來做視覺化
#!pip install wordcloud
for i in range(k):
    word_score = []
    for ind, score in zip(order_centroids[i, :10],  km.cluster_centers_[i, ::-1]):
        word_score.append((terms[ind], score))

word_score